Brian Goldsmith -brian.goldsmith@gmail.com - https://github.com/bdg221

# Task 2 find negatives values:

Given a list of integer numbers, look for a negative number in the list. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case


def find_negative_numbers(list[int]:list_number):
     “””
list_number : integer list!.
Return the True or False depends of the input
     “””

     # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc.

      # consider print your quantum circuit,


## Initial Plan

The first step will be to encode the list into a quantum state with qubits. If this was only dealing with non-negative numbers there are many methods for encoding the state vector including with indexes like (https://github.com/Qiskit/qiskit/issues/8290). However, this is for non-negative numbers and the task is specifically for negative numbers. Also, since the task returns True if a negative number is in the list, the indexes do not matter and do not need to be used.

I can think of two possible approaches for embedding the numbers:
1. If the numbers are embedded and the negative numbers have a negative phase applied to them, then Grover's algorithm can be used and it is as though the oracle has already been applied. This would then require amplitude amplification to change the difference in phase into a difference in amplitude. However, if all of the values in the list were positive or negative, then the resulting counts would be similar across all values. Therefore, it would be best to add a hardcoded negative value to the list. If the results after amplitude amplification show that at least two of the states are found to occur more likely, then True should be returned. If there is only a single state with an elevated count, then it is known the other values were all non-negative and False should be returned. After some thinking, it would make sense to make 0 have a negative phase since if zero is in the list it can never be negative.

2. Instead of using phase to indicate negative numbers, two's complement could be used instead. The following video shows implementing two's complement with quantum states: https://www.youtube.com/watch?v=NPhbjcaCfGs If manually embedding each number in the list, this would be much easier to accomplish than having to handle the phase. However, this does require an extra qubit and most frameworks have built-in functions to setup statevectors to any state. After the embedding is done, the most significant qubit could be measured and that would indicate if any of the numbers were negative with 1 returning True and 0 returning False.

In either scenario the embedding of the list would be where the most amount of work would be required. In simulations and easy examples, using the additional qubit with two's compliment would be simpler to implement. However, with a real world device, using less qubits would probably be optimal even though it requires more gates and a more complex circuit to handle amplitude amplification.

## OPTION 1 - Negative Phase and Amplitude Amplification

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector
from collections import Counter
import math
from qiskit import BasicAer
from qiskit.algorithms import AmplificationProblem
from qiskit.algorithms import Grover
from qiskit.primitives import Sampler

### Embed the list of numbers into a quantum state
The initial_state_vector method is being used to create the initial quantum state based off the provided list. This was the first stumbling block that I hit. After a great deal of research, I found that Qiskit allows for an initialize_state in the QuantumCircuit.initialize function. I must admit it feels a bit cheap to be manually constructing a state vector, along with a negative phase for negative numbers. However, I did not find a great alternative despite heavily researching the topic, including using Microsoft's Code with Azure Quantum that includes Copilot in Azure Quantum.

In [ ]:
def initial_state_vector(list_number: list[int], num_qubits: int) -> tuple[list[float], list[str]]:
    """
    This function creates the initial state vector based off using the numbers in list_number. During the process, a vector of bit strings named good_states is created with each negative number's absolute value appended in bit string form.

    :param list_number: List of values to be used for generating the initial state vector
    :param num_qubits: The number of qubits to be used by circuit
    :return: A list of floats that represents the initial state vector and a list of bit strings representing negative numbers
    """
    # use the length of the binary value of the max absolute value to get the number of states
    # this will be used for setting up the initial state vector
    num_states = 2 ** num_qubits

    # find the number of unique absolute values in the list to use for normalization
    # note that if the same value is positive and negative in the list, we will only care about the negative
    # also, +1 is added because the zero state is included with a negative phase
    normalize_val = len(Counter([abs(ele) for ele in list_number]).keys())+1

    # initialize the desired state vector to zeroes to start
    desired_vector = [0] * num_states

    # Set the zero state to have a negative phase to show a differentiator when the list contains all positive or all negatives values
    zero = '0' * num_qubits
    # initialize the "good_states" to include the zero state. The good_states is used by the Qiskit AmplificationProblem function.
    good_states = [zero]

    # Set the zero state in the desired_vector to have a negative phase
    desired_vector[0] = -1 / math.sqrt(normalize_val) * complex(1, 0)

    # loop through the provided list and modify the desired state vector
    for i in list_number:

        # if the value from the list is negative, set the phase to -
        if(i < 0):
            desired_vector[abs(i)] = -1 / math.sqrt(normalize_val) * complex(1, 0)
            good_states.append(format(abs(i), '0'+str(num_qubits)))

        # else if it is positive and a negative value doesn't already exist, set teh phase to +
        else:
            if(desired_vector[i] == 0):
                desired_vector[i] = 1 / math.sqrt(normalize_val) * complex(1, 0)

    return desired_vector, good_states

### Diffuse / Amplitude Amplification
The second step will be to use a diffuser, amplitude amplification, to change difference in phase to difference in amplitude. Since I am not looking for an exact answer, I am assuming a single application of the diffuser will be sufficient. The negative numbers already have a negative phase, so implementing a diffuser should boost their amplitudes resulting in the negative numbers have higher counts in the results. Since the zero state has a negative phase, it can be used as a rough guideline for negative numbers. If only the zero state has a higher number of results, then False can be returned. However, if any other state has similar results to the zero state, then True can be returned.

Again, the zero state is given a negative phase to differentiate the results when all of numbers in the list are positive or negative. If zero was not given a negative phase, both of those scenarios would result in the counts being roughly the same for every state.

Implementing the diffuser was another stumbling block in this task. While Qiskit has a built-in AmplificationProblem class (https://qiskit.org/documentation/stubs/qiskit.algorithms.AmplificationProblem.html), it does not seem possible to implement it without an oracle. However, due to the initial embedding of the numbers into the quantum state, an oracle is not needed. Trying to setup a dummy oracle with the state_preparation parameter set to the initialized state results in an error "Invalid param type <class 'complex'> for gate initialize_dg." which appears to be related to how the quantum state is initialized from the state vector. Changing the parameters to use the initialized state as the oracle did not result in an error, however the results were incorrect. So while I was hoping to utilize the built-in functionality to only apply the diffuser, this appears to not be feasible. Interestingly, using the incorrect results to determine whether to return True or False is successful for a number of instances, however it is not consistent.

In [ ]:
def find_negative_numbers(list_number: list[int] ) -> bool:
    """
    Take a list of integers and then embed the values into a single quantum system.

    NOTES: This function is condensing the values in list_number to unique absolute values with the negative values taking priority over positive values. For example, if [1, -1, 2] is provided, the desired_state vector will be -1/sqrt(2) |01> + 1/sqrt(2) |10>

    For doing the arbitrary initialization, the following example was used: https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits/3_summary_of_quantum_operations.ipynb

    :param list_number: integer list
    :return: True or False based on if list_number contains a negative number
    """

    #get maximum absolute value from the list to determine the number of qubits necessary
    maxVal = abs(max(list_number, key=abs))

    # number of qubits
    num_qubits = len("{0:b}".format(maxVal))

    # debug for maxVal and num_qubits
    # print("maxVal = {} and num_qubits = {}".format(str(maxVal), str(num_qubits)))

    # call initial_state_vector to generate the list that can be used when initializing the circuit, as well as getting a
    # list of bit strings with negative phases
    initial_state, good_states = initial_state_vector(list_number, num_qubits)

    # debug for initial state
    #print("Initial state vector: {}".format(initial_state))

    # create the QuantumCircuit
    q = QuantumRegister(num_qubits)
    qc = QuantumCircuit(q)
    qc.initialize(initial_state, q)

    # used to display the circuit with the initialized states
    #qc.draw()
    #print(qc)

    # attempt to use the built-in AmplificationProblem and Grover classes
    problem = AmplificationProblem(qc, is_good_state=good_states)

    grover = Grover(sampler=Sampler())
    result = grover.amplify(problem)
    counts = result.circuit_results

    # debug print results
    #print(counts)

    # get the zero state value since it is
    zero = '0' * num_qubits
    zero_result = counts[0][zero]

    # check results for other states against the zero state
    for key in counts[0]:

        # debug showing reuslts of other states
        print("State |{}> : {}".format(key, counts[0][key]))

        if key != zero and counts[0][key] >= zero_result:
            return True

    return False





In [ ]:
# test with 3 values
find_negative_numbers([3, -1, 5])

## Conclusion
This task was presented in the document as a simple task, yet I found it to be quite challenging. The largest problem that I faced is that when it comes to Grover's and Amplitude Amplification, searching online mostly results in the same 2-qubit toy example using a simple oracle of a CZ. With only this single example, it is extremely difficult to generalize the formulas and concepts from the paper. I found it very frustrating that I knew what I wanted to do, but couldn't find the appropriate resources to help guide me to how to write the code to accomplish it. For example, I thought of writing my own diffuser, but I was not sure how to generally apply a Z gate to all the non-"good_states". This is my motivation for applying for this program. I am hoping with some guidance I will be able to not only get a more complete understanding of concepts, but be able to take my ideas and bring them to life.